In [76]:
import networkx as nx
import random as r
import operator


def weight_calc(num_edges):           ##calculates weight edges based on number of edges contracted together
    return .9 ** num_edges ** 3       ##will cause rounding errors if numbers get too small (will who up as node not present error)


#input graph G and number of desired districts k
#outputs a k partition of G respresented as a list of lists
def agglom(G,k):    
    for n in (G.nodes()):  #initializes node attributes which track contractions
        G.node[n]['seg'] = [n]
    edges = {}
    for ed in (list(set(G.edges()) | set(nx.non_edges(G)))):  #initializes edge attributes which track 
        edges[ed] = 1                                         #edge contractions, which informs edge weighting
    while len(G.nodes) > k and len(G.edges) > 0 :
        weight_total = 0
        for edge in edges:          #calculates sum of edge weights (weights are calculated dynamically based on 
            if edges[edge] != 0:
                weight_total += weight_calc(edges[edge])  #number of edges contracted together)
        weight_total = r.uniform(0,weight_total)
        for edge in edges:
            if edges[edge] != 0 : weight_total -= weight_calc(edges[edge])      #random 'index' is chosen corresponding edge is found
            if weight_total <= 0:                                             #by subtracting edge weights until total reaches 0
                e = edge
                break
        inter = list(set(G.neighbors(e[0])) & set(G.neighbors(e[1])))       #looks for shared neighobors to measure edge contraction
        for shared in inter:
            if shared > e[1] : edges[(e[0],shared)] += edges[(e[1],shared)]  #recording edge contraction
            elif shared < e[0]: edges[(shared,e[0])] += edges[(shared,e[1])]
            else : edges[(e[0],shared)] += edges[(shared,e[1])]
        for n in G.neighbors(e[1]):
            if n < e[1] : edges[(n,e[1])] = 0
            else : edges[(e[1],n)] = 0    
        G.node[e[0]]['seg'] += (G.node[e[1]]['seg'])      #recording node contraction
        G = nx.contracted_edge(G, e, self_loops = False)  #contraction occurs
    result = nx.get_node_attributes(G,'seg')
    out = list()
    for w in result:
        out.append(sorted(result[w]))  #formatting output
    return out

In [77]:
#Example: 
G = nx.Graph()
G.add_nodes_from(range(0,5)) #add nodes
G.add_edges_from(nx.non_edges(G))
print (agglom(G,2))

[[0, 1, 4], [2, 3]]
